In [7]:
%reload_ext autoreload
%autoreload 2

In [8]:
import sys, os
proj = os.path.dirname(os.getcwd())
if proj not in sys.path:
    sys.path.insert(0, proj)
sys.path

['/Volumes/Projects 1/Evidently/record-thing',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python311.zip',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages']

In [10]:
# Device independent code
import torch

device = "cpu" 
if torch.cuda.is_available():
    device = "cuda"
# if torch.backends.mps.is_available:
#     device = torch.device("mps")
torch.device(device)
torch.set_default_device(device=device)

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [16]:
!docker compose up -d

/Users/henrikvendelbo/.zshenv:.:1: no such file or directory: /Users/henrikvendelbo/.cargo/env
[+] Running 1/0
 ✔ Container rqlite_server  Running                                        0.0s 


# UPC/QR lookup

If the scanner recognises a barcode or QR code the product number should be looked up.
If multiple matches are found the category of product should be used to pick the most likely match.
The user is asked to confirm the match YES/NO.

In [12]:
from dataset.product import lookup_upc
from dataset.rql import connection
from dataset.ingestion import Ingestor

upc = '0885909950805'
with connection.cursor() as cursor:
    ingestor = Ingestor(cursor, device=device)

    result = lookup_upc(upc, ingestor=ingestor)
    if result:
        print(result.as_dict())

        # match with product["items"][0]["images"] product similarity with pictures of the recording.
        # print(number_matches, upc, item)

Using cache found in /Volumes/Projects 1/Evidently/record-thing/.cache/torch/hub/facebookresearch_dinov2_main
/Volumes/Projects 1/Evidently/record-thing/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Volumes/Projects 1/Evidently/record-thing/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Volumes/Projects 1/Evidently/record-thing/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
from typing import Sequence
import torch
import requests

def generate_embedding(image: bytes) -> torch.Tensor:
    # dino.
    pass

def get_images(image_urls: Sequence[str]) -> Sequence[bytes]:
    images: Sequence[bytes] = [requests.get(url).content for url in image_urls]
    embeddings: Sequence[torch.Tensor] = [generate_embedding(image) for image in images]